<a href="https://colab.research.google.com/github/msjun23/Deep-Learning-from-Scratch/blob/main/Chapter3/MNIST_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#손글씨 숫자 인식
이미 학습된 매개변수를 사용하여 학습 과정은 생략하고, 추론 과정만 구현해본다. 이 추론 과정을 신경망의 **순전파**(forward propagation)라고 한다.

> 신경망을 이용해 문제를 해결하는 것은 두 단계로 나뉜다. 먼저 훈련 데이터(학습 데이터)를 사용해 가중치 매개변수를 학습하고, 추론 단계에서는 앞서 학습한 매개변수를 사용하여 입력 데이터를 분류한다.

#MNIST 데이터 셋

딥러닝을 한번이라도 공부해 봤다면 들어봤을 손글씨 숫자 이미지 집합이다. 아주 유명한 데이터셋으로 여러 논문 등에서 실험용 데이터로 많이 쓰인다.

0 부터 9 까지의 숫자 이미지로, 훈련(train) 데이터가 60,000장, 시험(test) 데이터가 10,000장 준비되어 있다. 각 이미지 데이터는 $28\times28$ 크기의 그레이스캐일(grayscale, 1차원) 이미지이며, 각 픽셀은 0 에서 255 까지의 값을 취한다.

In [1]:
import sys, os
sys.path.append(os.pardir)    # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from mnist import load_mnist

# 처음 한 번은 몇 분 정도 걸립니다.
(x_train, y_train), (x_test, y_test) = load_mnist(flatten=True, normalize=False)

# 각 데이터의 형상 출력
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


load_mnist라는 함수를 사용해서 MNIST 데이터를 (훈련 이미지, 훈련 레이블), (시험 이미지, 시험 레이블) 형식으로 불러온다. 인수로는 아래 세 가지를 설정할 수 있다. 모두 bool형으로 True로 설정하면 아래와 같다.

- normalize : 입력 이미지의 픽셀값을 0.0 ~ 1.0 사이의 값으로 정규화한다.
- flatten : 입력 이미지를 1차원 배열로 만든다. $1\times28\times28$의 3차원 배열을 784개의 원소를 가진 1차원 배열로 변환하여 저장한다.
- one_hot_label : 원-핫 인코딩이란, 정답 레이블 중 정답을 뜻하는 원소만 1이고 나머지는 모두 0인 배열이다. 정답 레이블의 형태를 원-핫 인코딩으로 저장한다.

In [2]:
# 첫 번째 훈련 이미지를 화면에 표시한다.
import sys, os
sys.path.append(os.pardir)
import numpy as np
from mnist import load_mnist
from PIL import Image

def img_show(img):
  pil_img = Image.fromarray(np.uint8(img))
  pil_img.show()

(x_train, y_train), (x_test, y_test) = load_mnist(flatten=True, normalize=False)

img = x_train[0]
label = y_train[0]
print(label)

print(img.shape)          # flatten=True : (784,) 형태의 1차원 배열
img = img.reshape(28, 28) # 원래 이미지의 모양으로 변형 (1*28*28)
print(img.shape)          # (28, 28)

img_show(img)

5
(784,)
(28, 28)


이미지를 표시하기 위해 넘파이로 표시된 이미지 데이터를 PIL용 객체로 변환해야 하며, 이 변환은 Image.fromarray()가 수행한다.

---



#신경망 추론 정리

MNIST 데이터셋을 가지고 추론을 수행하는 신경망을 구현할 차례이다. 입력 이미지의 크기가 $1\times28\times28$ 이므로 입력층 뉴런을 784개, 0 부터 9까지의 숫자를 분류해야 하기 때문에 출력층의 뉴런은 10개로 한다.

은닉층은 총 두개로, 첫 번째 은닉층에는 50개의 뉴런을, 두 번째 은닉층에는 100개의 뉴런을 배치한다. 이 수는 임의로 정한 값이다.

In [3]:
# sigmoid
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

# softmax
def softmax(a):
  c = np.max(a)
  exp_a = np.exp(a - c)
  sum_exp_a = np.sum(exp_a)
  y = exp_a / sum_exp_a

  return y

In [4]:
# 함수 정의
import pickle

def get_data():
  (x_train, y_train), (x_test, y_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
  return x_test, y_test

def init_network():
  with open('sample_weight.pkl', 'rb') as f:
    network = pickle.load(f)

  return network

def predict(network, x):
  W1, W2, W3 = network['W1'], network['W2'], network['W3']
  b1, b2, b3 = network['b1'], network['b2'], network['b3']

  a1 = np.dot(x, W1) + b1
  z1 = sigmoid(a1)
  a2 = np.dot(z1, W2) + b2
  z2 = sigmoid(a2)
  a3 = np.dot(z2, W3) + b3
  y = softmax(a3)

  return y

In [5]:
# 세 함수를 이용해 신경망에 의한 추론을 수행, 정확도(accuracy - 분류가 얼마나 올바른가) 평가
x_test, y_test = get_data()
network = init_network()

accuracy_cnt = 0
for i in range(len(x_test)):     # 10,000
  y = predict(network, x_test[i])
  p = np.argmax(y)    # 확률이 가장 높은 원소의 인덱스를 얻는다. 
  if p == y_test[i]:
    accuracy_cnt += 1

print('Accuracy: ' + str(float(accuracy_cnt) / len(x_test)))

Accuracy: 0.9352


0. 가장 먼저 MNIST 데이터셋을 얻고 네트워크를 생성한다.
1. for문을 돌며 x_test에 저장된 이미지를 한 장씩 꺼내 predict()함수로 분류한다.
2. 각 레이블의 확률이 반환되면, 이 배열에서 값이 가장 큰 원소의 인덱스를 구한다. 이것이 곧 예측 결과이다.
3. 예측한 답변과 정답 레이블을 비교하여 맞힌 숫자(accuracy_cnt)를 세고, 전체 이미지 숫자로 나눠 정확도를 구한다.

해당 예제에서 load_mnist 함수의 인수인 normalize를 True로 설정했다. 0 ~ 255 범위인 각 픽셀을 0.0 ~ 1.0 사이의 범위로 변환했다. 이처럼 데이터를 특정 범위로 변환하는 것을 **정규화**(normalization)라 하고, 신경망의 입력 데이터에 특정 변환을 가하는 것을 **전처리**(pre-processing)라 한다.

#배치 처리

In [6]:
x, _ = get_data()
network = init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3']

print(x.shape)
print(x[0].shape)
print(W1.shape)
print(W2.shape)
print(W3.shape)

(10000, 784)
(784,)
(784, 50)
(50, 100)
(100, 10)


위 결과에서 다차원 배열의 대응하는 차원의 원소 수가 일치함을 확인할 수 있다.

- X(784, ) - W1(784, 50) - W2(50, 100) - W3(100, 10) -> Y(10,)
> 이는 이미지 데이터를 1장만 입력했을 때의 처리 흐름이다.

이미지 여러 장을 한꺼번에 입력하는 경우를 생각해보자. 가령 이미지 100개를 묶어 predict() 함수에 한 번에 넘기면 아래와 같다.

- X(100, 784) - W1(784, 50) - W2(50, 100) - W3(100, 10) -> Y(100, 10)
> 100장 분량의 입력 데이터의 결과가 한 번에 출력됨을 나타낸다. x[0]와 y[0]에는 0번째 이미지와 그 추론 결과가, x[1]과 y[1]에는 1번째 이미지와 그 추론 결과가 저장되는 식이다.

이처럼 하나로 묶은 입력 데이터를 **배치**(batch)라고 한다.

배치 처리는 컴퓨터로 계산할 때 큰 이점을 준다. 그 이유는 아래와 같이 두가지가 있다.

1. 수치 계산 라이브러리는 대부분이 큰 배열을 효율적으로 처리할 수 있도록 최적화 되어있다.
2. 데이터 전송이 병목으로 작용하는 경우가 자주 있는데, 배치 처리를 함으로써 버스에 주는 부하를 줄일 수 있다.(느린 I/O를 통해 데이터를 읽는 횟수가 줄고, 빠른 CPU나 GPU로 순수 계산을 수행하는 비율이 높아진다.)

즉, 컴퓨터에서는 **큰 배열을 한꺼번에 계산**하는 것이 분할된 작은 배열을 여러 번 계산하는 것보다 빠르다.

In [7]:
# 배치 처리 적용
x_test, y_test = get_data()
network = init_network()

batch_size = 100    # 배치 크기
accuracy_cnt = 0

for i in range(0, len(x_test), batch_size):
  x_batch = x_test[i: i + batch_size]
  y_batch = predict(network, x_batch)
  p = np.argmax(y_batch, axis=1)
  accuracy_cnt += np.sum(p == y_test[i: i + batch_size])

print('Accuracy: ' + str(float(accuracy_cnt) / len(x)))

Accuracy: 0.9352


np.argmax() 함수를 사용하여 최댓값의 인덱스를 가져온다. 이때 axis=1 이라는 인수를 추가한 것에 주의한다. 이는 $100\times10$의 배열 중 1번째 차원을 구성하는 각 원소에서 (1번째 차원을 축으로)최댓값의 인덱스를 찾도록 한 것이다.(0번째 차원이 가장 처음 차원이다.) 여기에 대해서는 아래의 예시를 보면 이해가 빠를 것이다.

In [8]:
x = np.array([[0.1, 0.8, 0.1], [0.3, 0.1, 0.6], [0.2, 0.5, 0.3], [0.8, 0.1, 0.1]])
y = np.argmax(x, axis=1)
print(y)

[1 2 1 0]


마지막으로 배치 단위로 분류한 결과를 실제 답과 비교한다. 넘파이 배열끼리 비교하여 True / False로 구성된 bool 배열을 만들고, 이 결과에서 True의 수를 센다. 이 처리 과정은 아래 예식에서 확인할 수 있다.

In [9]:
y = np.array([1, 2, 1, 0])
t = np.array([1, 2, 0, 0])
print(y == t)
print(np.sum(y == t))

[ True  True False  True]
3
